<a href="https://colab.research.google.com/github/GabrieleCamara/Streamlit-EmotionalMaps/blob/main/Mapas_Emocionais_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Preparando o ambiente no Google Drive:

# Importando a biblioteca
from google.colab import drive

# Isso irá pedir sua autorização
drive.mount('/content/drive')

# Agora, seu Drive estará disponível em: /content/drive/My Drive

Mounted at /content/drive


In [ ]:
# Instalando as bibliotecas necessárias
!pip install streamlit
!pip install pyngrok
!pip install streamlit-folium
!pip install psycopg2-binary

In [232]:
# Importando bibliotecas para conexão com o banco de dados
from configparser import ConfigParser
import psycopg2
# Imporatando bibliotecas para a página do streamlit e para o mapa
import streamlit as st
from streamlit_folium import folium_static
import folium
# Biblioteca para os dados em geojson
import json

def config(filename='/content/drive/My Drive/Colab Notebooks/db-emotional_maps.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

conn = None
try:
  # read connection parameters
  params = config()
  # connect to the PostgreSQL server
  print('Connecting to the PostgreSQL database...')
  conn = psycopg2.connect(**params)
  # create a cursor
  cursor = conn.cursor()
  print ('Conexao realizada com sucesso')
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  print ('Erro na conexao com o banco de dados')


Connecting to the PostgreSQL database...
Conexao realizada com sucesso


In [ ]:
# Função que chama todos os nomes dos modais
def lista_vlc():
  lista_vlc = []
  cursor.execute("""SELECT DISTINCT ON (valencia) valencia FROM emoji_emoc;""")
  fet_list = cursor.fetchall()
  conn.commit()
  lista_vlc.append('Todas')
  for valencia in fet_list:
    lista_vlc.append(valencia[0])
  return lista_vlc

print(lista_vlc()[0])

Todas


In [ ]:
def func_pts_cnr():
	cursor.execute("""CREATE OR REPLACE VIEW pts_cnr_selec AS SELECT pts_cenarios.fid, pts_cenarios.pt_referencia, ST_Force2D(pts_cenarios.geom) AS geom 
	FROM pts_cenarios, cenarios
	WHERE pts_cenarios.cod_cenario = cenarios.cod_cenario AND cenarios.referencia = 'Parque Gomm - Praça 29 março'""")
	cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
	FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, pt_referencia) As l)) As properties FROM pts_cnr_selec As lg   ) As f )  As fc;""")
	json_pts_cnr = json.dumps(cursor.fetchall())
	conn.commit()
	

In [280]:
%%writefile app.py

# Importando bibliotecas para conexão com o banco de dados
from configparser import ConfigParser
import psycopg2
# Imporatando bibliotecas para a página do streamlit e para o mapa
import streamlit as st
from streamlit_folium import folium_static
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import IFrame
# Biblioteca para os dados em geojson
import json

# --- CONEXÃO COM O BANCO DE DADOS ---#
def config(filename='/content/drive/My Drive/Colab Notebooks/db-emotional_maps.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

conn = None
try:
  # read connection parameters
  params = config()
  # connect to the PostgreSQL server
  print('Connecting to the PostgreSQL database...')
  conn = psycopg2.connect(**params)
  # create a cursor
  cursor = conn.cursor()
  print ('Conexao realizada com sucesso')
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  print ('Erro na conexao com o banco de dados')

m = folium.Map(location = [-25.43,-49.26], zoom_start = 13, tiles = 'Cartodb Positron', attr = '© contribuidores do OpenStreetMap (CC BY-SA 2.0)')

# -- FUNÇÕES --- #
# Função que chama todos os nomes das emoções
def lista_emoc():
  lista_emoc = []
  cursor.execute("""SELECT emocao FROM emoji_emoc;""")
  fet_list = cursor.fetchall()
  conn.commit()
  lista_emoc.append('')
  for emocao in fet_list:
    lista_emoc.append(emocao[0])
  return lista_emoc

# Função que chama todos os nomes dos modais
def lista_mdl():
  lista_mdl = []
  cursor.execute("""SELECT nome FROM modais;""")
  fet_list = cursor.fetchall()
  conn.commit()
  lista_mdl.append('')
  for nome in fet_list:
    lista_mdl.append(nome[0])
  return lista_mdl

# Função que chama todos os nomes dos cenarios
def lista_cnr():
  lista_cnr = []
  cursor.execute("""SELECT referencia FROM cenarios;""")
  fet_list = cursor.fetchall()
  conn.commit()
  lista_cnr.append('')
  for referencia in fet_list:
    lista_cnr.append(referencia[0])
  return lista_cnr

# Função que chama todos as faixa etárias dos participantes
def lista_etr():
  lista_etr = []
  cursor.execute("""SELECT DISTINCT ON (faixa_etaria) faixa_etaria FROM participantes;""")
  fet_list = cursor.fetchall()
  conn.commit()
  lista_etr.append('')
  for faixa_etaria in fet_list:
    lista_etr.append(faixa_etaria[0])
  return lista_etr

# Função que chama os generos dos participantes
def lista_gnr():
  lista_gnr = []
  cursor.execute("""SELECT DISTINCT ON (genero) genero FROM participantes;""")
  fet_list = cursor.fetchall()
  conn.commit()
  lista_gnr.append('')
  for genero in fet_list:
    lista_gnr.append(genero[0])
  return lista_gnr

# Função que chama todos as valencias das emoções
def lista_vlc():
  lista_vlc = []
  cursor.execute("""SELECT DISTINCT ON (valencia) valencia FROM emoji_emoc;""")
  fet_list = cursor.fetchall()
  conn.commit()
  for valencia in fet_list:
    lista_vlc.append(valencia[0])
  return lista_vlc

# --- CONSULTAS NO BANCO --- #
# Função que executa as consultas EMOÇÕES PONTO individuais
def func_emoc_indiv(emoc_indiv):
	cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_indiv AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom 
  FROM emocoes_coletadas, emoji_emoc WHERE emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND emocao = '%s'""" %emoc_indiv)
	cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features 
	FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_indiv As lg   ) As f )  As fc;""")
	json_emoc_indiv = json.dumps(cursor.fetchall())
	conn.commit()
	# st.json(json_emoc_indiv)
  # this is the layer that eventually gets added to Map
	layer = folium.FeatureGroup(name='Emoção: %s'%emoc_indiv,)
	gj = folium.GeoJson(json_emoc_indiv[2:len(json_emoc_indiv)-2])
	emoc_mkr = []
	# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
	for feature in gj.data['features']:
		icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
		icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
		emoc_mkr.append(list(reversed(feature['geometry']['coordinates'])))
		if feature['geometry']['type'] == 'Point':
			folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
												icon=icon).add_to(layer)
	layer.add_to(m)
  # Mapa de calor das emoções individuais
	hm_emoc_indiv = folium.plugins.HeatMap(emoc_mkr, name = 'Mapa de calor da emoção: %s'%emoc_indiv, radius=40, blur=15, overlay=True, control=True, show=False)
	hm_emoc_indiv.add_to(m)
 
# Função que executa as consultas EMOÇÕES PONTO modais
def func_emoc_mdl(emoc_mdl, emoc_vlc):
	if len(emoc_vlc) == 3:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_mdl AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, modais, emoji_emoc
		WHERE emocoes_coletadas.cod_modal = modais.cod_modal AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND modais.nome = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, modais, emoji_emoc
		WHERE emocoes_coletadas.cod_modal = modais.cod_modal AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND modais.nome = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, modais, emoji_emoc
		WHERE emocoes_coletadas.cod_modal = modais.cod_modal AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND modais.nome = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_mdl, emoc_vlc[0], emoc_mdl, emoc_vlc[1], emoc_mdl, emoc_vlc[2]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  	FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_mdl As lg   ) As f )  As fc;""")
		json_emoc_mdl = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções de todas as valências do modal: %s' %emoc_mdl,)
		gj = folium.GeoJson(json_emoc_mdl[2:len(json_emoc_mdl)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
													icon=icon).add_to(layer)
		layer.add_to(m)
	elif len(emoc_vlc) == 2:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_mdl AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, modais, emoji_emoc
		WHERE emocoes_coletadas.cod_modal = modais.cod_modal AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND modais.nome = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, modais, emoji_emoc
		WHERE emocoes_coletadas.cod_modal = modais.cod_modal AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND modais.nome = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_mdl, emoc_vlc[0], emoc_mdl, emoc_vlc[1]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  	FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_mdl As lg   ) As f )  As fc;""")
		json_emoc_mdl = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções das valências %s e %s do modal: %s' %(emoc_vlc[0], emoc_vlc[1], emoc_mdl),)
		gj = folium.GeoJson(json_emoc_mdl[2:len(json_emoc_mdl)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
													icon=icon).add_to(layer)
		layer.add_to(m)
	elif len(emoc_vlc) == 1:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_mdl AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, modais, emoji_emoc
		WHERE emocoes_coletadas.cod_modal = modais.cod_modal AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND modais.nome = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_mdl, emoc_vlc[0]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  	FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_mdl As lg   ) As f )  As fc;""")
		json_emoc_mdl = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções da valência %s do modal: %s' %(emoc_vlc[0], emoc_mdl),)
		gj = folium.GeoJson(json_emoc_mdl[2:len(json_emoc_mdl)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
													icon=icon).add_to(layer)
		layer.add_to(m)
	else:
		st.info('Escolha uma os mais valências para visualizar as emoções filtradas por modal')
 
# Função que executa as consultas EMOÇÕES PONTO cenários
def func_emoc_cnr(emoc_cnr, emoc_vlc):
	if len(emoc_vlc) == 3:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_cnr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, cenarios, emoji_emoc
		WHERE emocoes_coletadas.cod_cenario = cenarios.cod_cenario AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND cenarios.referencia = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, cenarios, emoji_emoc
		WHERE emocoes_coletadas.cod_cenario = cenarios.cod_cenario AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND cenarios.referencia = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, cenarios, emoji_emoc
		WHERE emocoes_coletadas.cod_cenario = cenarios.cod_cenario AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND cenarios.referencia = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_cnr, emoc_vlc[0], emoc_cnr, emoc_vlc[1], emoc_cnr, emoc_vlc[2]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
		FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_cnr As lg   ) As f )  As fc;""")
		json_emoc_cnr = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções de todas as valências do cenário: %s' %emoc_cnr,)
		gj = folium.GeoJson(json_emoc_cnr[2:len(json_emoc_cnr)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
												icon=icon).add_to(layer)
		layer.add_to(m)
	elif len(emoc_vlc) == 2:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_cnr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, cenarios, emoji_emoc
		WHERE emocoes_coletadas.cod_cenario = cenarios.cod_cenario AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND cenarios.referencia = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, cenarios, emoji_emoc
		WHERE emocoes_coletadas.cod_cenario = cenarios.cod_cenario AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND cenarios.referencia = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_cnr, emoc_vlc[0], emoc_cnr, emoc_vlc[1]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
		FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_cnr As lg   ) As f )  As fc;""")
		json_emoc_cnr = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções das valências %s e %s do cenário: %s' %(emoc_vlc[0], emoc_vlc[1], emoc_cnr),)
		gj = folium.GeoJson(json_emoc_cnr[2:len(json_emoc_cnr)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
												icon=icon).add_to(layer)
		layer.add_to(m)
	elif len(emoc_vlc) == 1:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_cnr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, cenarios, emoji_emoc
		WHERE emocoes_coletadas.cod_cenario = cenarios.cod_cenario AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND cenarios.referencia = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_cnr, emoc_vlc[0]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
		FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_cnr As lg   ) As f )  As fc;""")
		json_emoc_cnr = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções da valencia %s do cenário: %s' %(emoc_vlc[0], emoc_cnr),)
		gj = folium.GeoJson(json_emoc_cnr[2:len(json_emoc_cnr)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
												icon=icon).add_to(layer)
		layer.add_to(m)
	else:
		st.info('Escolha uma os mais valências para visualizar as emoções nofiltradas por cenário')

# Função que executa as consultas para mostrar os pontos dos cenário
def func_pts_cnr(emoc_cnr):
	cursor.execute("""CREATE OR REPLACE VIEW pts_cnr_selec AS SELECT pts_cenarios.fid, pts_cenarios.pt_referencia, ST_Force2D(pts_cenarios.geom) AS geom 
	FROM pts_cenarios, cenarios
	WHERE pts_cenarios.cod_cenario = cenarios.cod_cenario AND cenarios.referencia = '%s'""" %emoc_cnr)
	cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
	FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, pt_referencia) As l)) As properties FROM pts_cnr_selec As lg   ) As f )  As fc;""")
	json_pts_cnr = json.dumps(cursor.fetchall())
	conn.commit()
	# st.json(json_pts_cnr)
  # this is the layer that eventually gets added to Map
	layer = folium.FeatureGroup(name='Pontos de referência do cenário',)
	gj = folium.GeoJson(json_pts_cnr[2:len(json_pts_cnr)-2])

	# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
	for feature in gj.data['features']:
		if feature['geometry']['type'] == 'Point':
			folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
			              popup = feature['properties']['pt_referencia'],
										icon = folium.Icon(color = "gray", icon = "ok",)
								 ).add_to(layer)
	layer.add_to(m)

# Função que executa as consultas EMOÇÕES PONTO faixa etária
def func_emoc_etr(emoc_etr, emoc_vlc):
	if len(emoc_vlc) == 3:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_etr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc
		WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.faixa_etaria = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc
		WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.faixa_etaria = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc 
		WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.faixa_etaria = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_etr, emoc_vlc[0], emoc_etr, emoc_vlc[1], emoc_etr, emoc_vlc[2]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
		FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_etr As lg   ) As f )  As fc;""")
		json_emoc_etr = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções de todas as valências da faixa etária: %s' %emoc_etr,)
		gj = folium.GeoJson(json_emoc_etr[2:len(json_emoc_etr)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])), icon=icon).add_to(layer)
		layer.add_to(m)
	elif len(emoc_vlc) == 2:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_etr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc
		WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.faixa_etaria = '%s' AND emoji_emoc.valencia = '%s'
		UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc 
		WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.faixa_etaria = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_etr, emoc_vlc[0], emoc_etr, emoc_vlc[1]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
		FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_etr As lg   ) As f )  As fc;""")
		json_emoc_etr = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções das valencias %s e %s da faixa etária: %s' %(emoc_vlc[0], emoc_vlc[1], emoc_etr),)
		gj = folium.GeoJson(json_emoc_etr[2:len(json_emoc_etr)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])), icon=icon).add_to(layer)
		layer.add_to(m)
	elif len(emoc_vlc) == 1:
		cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_etr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc
		WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.faixa_etaria = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_etr, emoc_vlc[0]))
		cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
		FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_etr As lg   ) As f )  As fc;""")
		json_emoc_etr = json.dumps(cursor.fetchall())
		conn.commit()
		# st.json(json_2)
  	# this is the layer that eventually gets added to Map
		layer = folium.FeatureGroup(name='Emoções da valência %s da faixa etária: %s' %(emoc_vlc[0], emoc_etr),)
		gj = folium.GeoJson(json_emoc_etr[2:len(json_emoc_etr)-2])

		# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
		for feature in gj.data['features']:
			icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
			icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
			if feature['geometry']['type'] == 'Point':
				folium.Marker(location=list(reversed(feature['geometry']['coordinates'])), icon=icon).add_to(layer)
		layer.add_to(m)
	else:
		st.info('Escolha uma os mais valências para visualizar as emoções filtradas por faixa etária dos participantes')
 
# Função que executa as consultas EMOÇÕES PONTO genero
# --- SEGUNDA VERSÃO INTERATIVA COM VALENCIA --- #
def func_emoc_gnr(emoc_gnr, emoc_vlc):
  if len(emoc_vlc) == 3:
    cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_gnr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia
    FROM emocoes_coletadas, participantes, emoji_emoc WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.genero = '%s' AND emoji_emoc.valencia = '%s' 
    UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc 
    WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.genero = '%s' AND emoji_emoc.valencia = '%s'
    UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc
    WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.genero = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_gnr, emoc_vlc[0], emoc_gnr, emoc_vlc[1], emoc_gnr, emoc_vlc[2]))
    cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_gnr As lg   ) As f )  As fc;""")
    json_emoc_gnr = json.dumps(cursor.fetchall())
    conn.commit()
    # st.json(json_2)
    # this is the layer that eventually gets added to Map
    layer = folium.FeatureGroup(name='Emoções de todas as valencias do gênero: %s' %emoc_gnr,)
    gj = folium.GeoJson(json_emoc_gnr[2:len(json_emoc_gnr)-2])

    # iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
    for feature in gj.data['features']:
      icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
      icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
      if feature['geometry']['type'] == 'Point':
        folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
				                  icon=icon).add_to(layer)
    layer.add_to(m)
  elif len(emoc_vlc) == 2:
    cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_gnr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia
    FROM emocoes_coletadas, participantes, emoji_emoc WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.genero = '%s' AND emoji_emoc.valencia = '%s' 
    UNION SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia FROM emocoes_coletadas, participantes, emoji_emoc 
    WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji AND participantes.genero = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_gnr, emoc_vlc[0], emoc_gnr, emoc_vlc[1]))
    cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_gnr As lg   ) As f )  As fc;""")
    json_emoc_gnr = json.dumps(cursor.fetchall())
    conn.commit()
    # st.json(json_2)
    # this is the layer that eventually gets added to Map
    layer = folium.FeatureGroup(name='Emoções de valencias %s e %s do gênero: %s' %(emoc_vlc[0], emoc_vlc[1], emoc_gnr),)
    gj = folium.GeoJson(json_emoc_gnr[2:len(json_emoc_gnr)-2])

    # iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
    for feature in gj.data['features']:
      icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
      icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
      if feature['geometry']['type'] == 'Point':
        folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
				                  icon=icon).add_to(layer)
    layer.add_to(m)
  elif len(emoc_vlc) == 1:
    cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_gnr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom, emoji_emoc.valencia
    FROM emocoes_coletadas, participantes, emoji_emoc WHERE emocoes_coletadas.cod_part = participantes.cod_part AND emocoes_coletadas.cod_emoji = emoji_emoc.cod_emoji
		AND participantes.genero = '%s' AND emoji_emoc.valencia = '%s'""" %(emoc_gnr, emoc_vlc[0]))
    cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_gnr As lg   ) As f )  As fc;""")
    json_emoc_gnr = json.dumps(cursor.fetchall())
    conn.commit()
    # st.json(json_2)
    # this is the layer that eventually gets added to Map
    layer = folium.FeatureGroup(name='Emoções de valencia %s do gênero: %s' %(emoc_vlc[0], emoc_gnr),)
    gj = folium.GeoJson(json_emoc_gnr[2:len(json_emoc_gnr)-2])

    # iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
    for feature in gj.data['features']:
      icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
      icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
      if feature['geometry']['type'] == 'Point':
        folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
				                  icon=icon).add_to(layer)
    layer.add_to(m)
  else:
    st.info('Escolha uma os mais valências para visualizar as emoções filtradas por gênero dos participantes')

# --- PÁGINA --- #
# Configurações da aba da página
PAGE_CONFIG = {"page_title":"Mapas Emocionais","page_icon":":heart:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

def main():
	# --- CANVA --- #
	st.title("Mapas Emocionais")
	# st.subheader("Baseado num caderno do Colab")
 
  # --- SIDEBAR --- #
	st.sidebar.subheader("Consulta Pontuais")  
	emoc_indiv = st.sidebar.selectbox('Escolha uma emoção', lista_emoc())
	emoc_mdl = st.sidebar.selectbox('Escolha um meio de transporte', lista_mdl())
	emoc_cnr = st.sidebar.selectbox('Escolha um cenário', lista_cnr())
	st.sidebar.subheader("Consulta Pontuais de acordo com o perfil do usuário")
	emoc_etr = st.sidebar.selectbox('Escolha uma faixa etária', lista_etr())
	emoc_gnr = st.sidebar.selectbox('Escolha um gênero', lista_gnr()) 
	emoc_vlc = st.multiselect('Escolha as valências da emoções consultadas', lista_vlc(), default=lista_vlc())

  # Executa a ação no selectbox emoção 
	if emoc_indiv != '' :
		func_emoc_indiv(emoc_indiv)
	else: 
		pass
  
  # Executa a ação no selectbox modal 
	if emoc_mdl != '':
		func_emoc_mdl(emoc_mdl, emoc_vlc)
	else:
		pass

  # Executa a ação no selectbox cenarios
	if emoc_cnr != '':
		# emoc_vlc = st.multiselect('Escolha as valências da emoções consultadas', lista_vlc(), default=lista_vlc())
		func_emoc_cnr(emoc_cnr, emoc_vlc)
		func_pts_cnr(emoc_cnr)
	else:
		pass

  # Executa a ação no selectbox faixa etaria
	if emoc_etr != '':
		# emoc_vlc = st.multiselect('Escolha as valências da emoções consultadas', lista_vlc(), default=lista_vlc())
		func_emoc_etr(emoc_etr, emoc_vlc)
	else:
		pass

  # Executa a ação no selectbox genero
	if emoc_gnr != '':
		# emoc_vlc = st.multiselect('Escolha as valências da emoções consultadas', lista_vlc(), default=lista_vlc())
		# Executa a função com a consulta espacial passando os dois parâmetros
		func_emoc_gnr(emoc_gnr, emoc_vlc)
	else:
		pass
  
  # --- MAPA DO CANVA --- #
	folium.LayerControl(collapsed = False,).add_to(m)
	folium_static(m)


if __name__ == '__main__':
	main()
 
cursor.close()
conn.close()

Overwriting app.py


In [281]:
!ls

app.py	drive  sample_data


In [282]:
  !streamlit run --server.port 80 app.py &>/dev/null&

In [273]:
from pyngrok import ngrok
!ngrok authtoken (1yBApN5G9ScuEKz89XAYSpi15EE_3WAXhK5XEqadmJRkA6DeB)
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `ngrok authtoken (1yBApN5G9ScuEKz89XAYSpi15EE_3WAXhK5XEqadmJRkA6DeB)'


2021-10-10 21:25:08.665 Opening tunnel named: http-80-1ec09e6b-2506-41c5-9b3b-ecdaac9c703b
2021-10-10 21:25:08.700 t=2021-10-10T21:25:08+0000 lvl=info msg="no configuration paths supplied"
2021-10-10 21:25:08.703 t=2021-10-10T21:25:08+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2021-10-10 21:25:08.705 t=2021-10-10T21:25:08+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2021-10-10 21:25:08.714 t=2021-10-10T21:25:08+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2021-10-10 21:25:08.911 t=2021-10-10T21:25:08+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2021-10-10 21:25:08.913 t=2021-10-10T21:25:08+0000 lvl=info msg="client session established" obj=csess id=0dcf5add5efa
2021-10-10 21:25:08.921 t=2021-10-10T21:25:08+0000 lvl=info msg=start pg=/api/tunnels id=11be76b4206c7abe
2021-10-10 21:25:08.923 t=2021-10-10T21:25:08+0000 lvl=info msg=end pg=/api/tunnels id=11be76b4206c7abe 

<NgrokTunnel: "http://77d3-34-73-237-172.ngrok.io" -> "http://localhost:80">

2021-10-10 21:25:09.017 t=2021-10-10T21:25:09+0000 lvl=info msg=end pg="/api/tunnels/http-80-1ec09e6b-2506-41c5-9b3b-ecdaac9c703b (http)" id=beb2b4a8d0cc88af status=200 dur=298.097µs


In [272]:
# mata o processso do ngork para reiniciar (se necessário)
from pyngrok import ngrok
ngrok.kill()

2021-10-10 21:25:05.757 Killing ngrok process: 3243


In [ ]:
# --- CONSULTAS ESPACIAIS FUNCIONANDO COM PARÂMETROS SEM A CLASSIFICAÇÃO DAS VALENCIAS --- #
# --- PRIMEIRA VERSÃO FUNCIONANDO --- #
# Função que executa as consultas EMOÇÕES PONTO genero
# def func_emoc_gnr(emoc_gnr):
# 	cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_gnr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom FROM emocoes_coletadas, participantes 
# 	WHERE emocoes_coletadas.cod_part = participantes.cod_part AND participantes.genero = '%s'""" %emoc_gnr)
# 	cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
#   FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_gnr As lg   ) As f )  As fc;""")
# 	json_emoc_gnr = json.dumps(cursor.fetchall())
# 	conn.commit()
# 	# st.json(json_2)
#   # this is the layer that eventually gets added to Map
# 	layer = folium.FeatureGroup(name='Emoções do gênero: %s' %emoc_gnr,)
# 	gj = folium.GeoJson(json_emoc_gnr[2:len(json_emoc_gnr)-2])

# 	# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
# 	for feature in gj.data['features']:
# 		icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
# 		icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
# 		if feature['geometry']['type'] == 'Point':
# 			folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
# 												icon=icon).add_to(layer)
# 	layer.add_to(m)

# Função que executa as consultas EMOÇÕES PONTO faixa etaria
# def func_emoc_etr(emoc_etr):
# 	cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_etr AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom FROM emocoes_coletadas, participantes
#   WHERE emocoes_coletadas.cod_part = participantes.cod_part AND participantes.faixa_etaria = '%s'""" %emoc_etr)
# 	cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
#   FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_etr As lg   ) As f )  As fc;""")
# 	json_emoc_etr = json.dumps(cursor.fetchall())
# 	conn.commit()
# 	# st.json(json_2)
#   # this is the layer that eventually gets added to Map
# 	layer = folium.FeatureGroup(name='Emoções da faixa etária: %s' %emoc_etr,)
# 	gj = folium.GeoJson(json_emoc_etr[2:len(json_emoc_etr)-2])

# 	# iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
# 	for feature in gj.data['features']:
# 		icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
# 		icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
# 		if feature['geometry']['type'] == 'Point':
# 			folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
# 												icon=icon).add_to(layer)
# 	layer.add_to(m)

# Função que executa as consultas EMOÇÕES PONTO modais
# def func_emoc_mdl(emoc_mdl):
	# cursor.execute("""CREATE OR REPLACE VIEW emoc_colec_mdl AS SELECT emocoes_coletadas.fid, emocoes_coletadas.cod_emoji, ST_Force2D(emocoes_coletadas.geom) AS geom FROM emocoes_coletadas, modais 
	# WHERE emocoes_coletadas.cod_modal = modais.cod_modal AND modais.nome = '%s'""" %emoc_mdl)
	# cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
  # FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom)::json As geometry, row_to_json((SELECT l FROM (SELECT fid, cod_emoji) As l)) As properties FROM emoc_colec_mdl As lg   ) As f )  As fc;""")
	# json_emoc_mdl = json.dumps(cursor.fetchall())
	# conn.commit()
	# # st.json(json_2)
  # # this is the layer that eventually gets added to Map
	# layer = folium.FeatureGroup(name='Emoções do modal: %s' %emoc_mdl,)
	# gj = folium.GeoJson(json_emoc_mdl[2:len(json_emoc_mdl)-2])

	# # iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
	# for feature in gj.data['features']:
	# 	icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/Streamlit-EmotionalMaps/main/Lista_Final_Emojis/%s.png' %feature['properties']['cod_emoji']
	# 	icon = folium.features.CustomIcon(icon_url, icon_size=(25, 25))
	# 	if feature['geometry']['type'] == 'Point':
	# 		folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
	# 											icon=icon).add_to(layer)
	# layer.add_to(m)